# Machine Learning Project
### By Pavan

## Introduction

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

import gym
from sklearn.preprocessing import KBinsDiscretizer
from IPython.display import clear_output

In [2]:
name = 'CartPole-v1'
env = gym.make(name)

The game: A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.

This game has a discrete action space but a continous state space. To form the Q-Table we must turn the continous state space into a discrete state space using a discretizer which uniformly divides the continous state space into a 2D grid of bins.

In [3]:
print('Action Space:', env.action_space) # Discrete action space
print('State Space:', env.observation_space) # Continous state space

Action Space: Discrete(2)
State Space: Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)


## Generating the Q-Table

### Setting the parameters

In [4]:
n_episodes = 2000

discount_rate = 0.99

min_exp_rate = 0.01
exp_decay_rate = 0.001

min_learn_rate = 0.01
learn_decay_rate = 0.001

### Creating the discretizer to uniformly divide the continous state into bins

In [5]:
n_bins = (6,12)

# Pole angle between the low and high from the environment (-24,24) degrees
# Pole angular velocity between (-50,50) degrees
lower_bounds = [env.observation_space.low[2], -np.radians(50)]
upper_bounds = [env.observation_space.high[2], np.radians(50)]

# Converts the continous state space to a discrete state space
def discretizer(_, __, angle, pole_velocity):
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds])
    return tuple(map(int,est.transform([[angle, pole_velocity]])[0]))

### Initializing the Q-Table

In [6]:
q_table = np.zeros(n_bins + (env.action_space.n,))
q_table.shape

(6, 12, 2)

The Q-Table has 6 bins for the angle, 12 bins for the angular velocity and 2 bins for the actions

### Creating helper functions

In [7]:
# Best action given the state
def best_action(state:tuple):
    return np.argmax(q_table[state])

In [8]:
# Learning rate decay
def learning_rate(e:int) -> float  :
    return min_learn_rate + (1 - min_learn_rate) * np.exp(-learn_decay_rate*e)

In [9]:
# Exploration rate decay
def exploration_rate(e:int) -> float :
    return min_exp_rate + (1 - min_exp_rate) * np.exp(-exp_decay_rate*e)

### Building the Q-Table

In [10]:
scores = []

for e in range(n_episodes):
    
    # Discretize state into buckets
    current_state = discretizer(*env.reset())
    
    done = False
    score = 0
    while done!=True:
        
        # Choose between exploration and exploitation
        er = exploration_rate(e)
        if np.random.random() <= er: 
            action = env.action_space.sample() # Explore
        else:
            action = best_action(current_state) # Exploit
        
        # Increment enviroment
        observation, reward, done, _ = env.step(action)
        new_state = discretizer(*observation)
        
        # Update Q-Table according to the Bellman equation
        lr = learning_rate(e)
        old_value = q_table[current_state][action]
        learned_value = reward + discount_rate*np.max(q_table[new_state])
        q_table[current_state][action] = (1-lr)*old_value + lr*learned_value
        
        # Update variables
        current_state = new_state
        score+=1
        env.render()
    
    print('Episode:', e)
    print('Score:', score)
    clear_output(wait=True)
    scores.append(score)

Episode: 1999
Score: 500


### Plotting the scores during learning

In [11]:
es = [e for e in range(n_episodes)]
plt.plot(es, scores)
plt.title('Scores')
plt.grid()
plt.show()

The scores are increasing over time which means it's learning

## Playing the game

In [18]:
# Play the game 100 times with the calculated Q-Table
scores = []
for e in range(100):
    observation = env.reset()
    state = discretizer(*observation)
    done = False
    score = 0
    while done!=True:
        env.render()
        action = best_action(state)
        observation, reward, done, info = env.step(action)
        state = discretizer(*observation)
        score+=1
    scores.append(score)
    print('Episode:', e)
    print('Score:', score)
    clear_output(wait=True)
env.close()
print('The average score is:', np.mean(scores))

The average score is: 500.0


Since 500 is the maximum score possible, the algorithm works well